<h1>MongoDB and JSON: Using Python to Insert Data</h1>
Data is pulled from the google maps api and inserted into MongoDB.

Before running this code, make sure you've started MongoDB from your shell and connected to it, otherwise your records won't be added.
<p>On Windows and Mac: https://docs.mongodb.com/manual/mongo/</p>
<p>On Linux: https://dzone.com/articles/mongodb-commands-cheat-sheet-for-beginners</p>

In this file we will:
<ol type = '1'>  
<li>Set things up by bringing in our dependencies and connecting to MongoDB </li>
<li>Insert records using an external JSON file</li>
<li>Insert records using an API call</li>
<li>Look up records by ID</li>
<li>Look up records by string search</li>
</ol>


<h3><u>Setup</u?</h3>

In [1]:
#Dependencies

##create tables
import pandas as pd 

##Tool for converting Zip Codes into Latitude and Longitude coordinates
import pgeocode 

##Use API key
from config import gkey, okey 

##API lookup
import requests

##Reading JSONS
import json 

##Connecting and using MongoDB commands in Python
import pymongo 

##Create artificial delay if lookups cause timeouts
import time 

In [20]:
#Connect to mongo db and create "locations_mdb" database
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

#Create "locations_mdb" database and assign it to a variable
#(note: MongoDB automatically creates a database when you call it, so this will do both)
db = client.locations_mdb

#Creating or swithcing to a collection
clients = db["clients"]  
hotels = db["hotels"]



<h2>Method 1: Insert Data into MongoDB from an External JSON file</h2>
<p>Using a randomly generated JSON of fictional clients, we will add records to our collection.</p>

In [15]:
#Read and Parse the JSON File
myfile=open("generated.json","r")
json_file=json.load(myfile)

#Load into MongoDB
db.hotels.insert_many(json_file)

<h2>Method 2: Insert Data Directly into MongoDB from the API Pull</h2>

In [35]:
# If you are looking up data from an API, you might have a table of values that you are using for your queries. Here, we will use zip code data to look up the locations of hotels in San Fransisco.

# Build the zip code reference table in PANDAS
zips_df=pd.read_csv('resources/zip_codes.csv')
zips_df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,ZIPCode,Type,County,Population,Area Code(s),Latitude,Longitude
0,0,0,0,94102,Standard,San Francisco,"31,176",415 / 510,37.7813,-122.4167
1,1,1,1,94103,Standard,San Francisco,"27,170",415 / 510 / 650,37.7725,-122.4147
2,2,2,2,94104,Standard,San Francisco,406,415 / 510 / 650 / 628,37.7915,-122.4018
3,3,3,3,94105,Standard,San Francisco,"5,846",415 / 510 / 650 / 628,37.7864,-122.3892
4,4,4,4,94107,Standard,San Francisco,"26,599",415 / 510 / 650,37.7621,-122.3971


In [28]:
#Create a collection ("hotels_full") in MongoDB and load in data from google api

#Save the api_url to a variable
api_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#Loop through the zip code data and...
 
#-->save each lat,long pair to a variable.
for index, row in zips_df.iterrows():
    #time.sleep(0.2) #--> This prevents timeouts from pulling API data too quickly.
    target_coordinates = f"{zips_df.loc[index, 'Latitude']},{zips_df.loc[index, 'Longitude']}" 
    target_radius = 50000
    target_type = "lodging"

#--> set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": gkey}

#--> Request a JSON using our parameters dictionary
    response = requests.get(api_url, params=params).json()
    
#-->Insert the record into MongoDB using our connected database and Mongo's "insert_one" function.
#Note: Remember that we created a variable "db" earlier to represent our database. That's where the "db" below comes from.
    for hotel in response["results"]:
        db.hotels.insert_one(hotel) 
    

In [7]:
#Build hotels database in PANDAS (to get rid of duplicates)
hotels_df = pd.DataFrame(columns = ['Name', 'Address', "Status", 'Lattitude', "Longitude"]) 

api_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in zips_df.iterrows():
    time.sleep(0.5)
    target_coordinates = f"{zips_df.loc[index, 'Latitude']},{zips_df.loc[index, 'Longitude']}" 
    target_radius = 50000
    target_type = "lodging"

# set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": gkey}

    response = requests.get(api_url, params=params).json()

# create a pandas dataframe of hotels
    for i in range(0,len(response["results"])):
        name=response["results"][i]["name"]
        lat=response["results"][i]["geometry"]["location"]["lat"]
        long=response["results"][i]["geometry"]["location"]["lng"]
        status=response["results"][i]["business_status"]
        address=response["results"][i]['vicinity']

        hotels_df = hotels_df.append({'Name' : name, 'Address' : address, 'Status' : status, "Latitude" :lat, "Longitude": long},  
                ignore_index = True) 
    

In [57]:
#Drop any duplicates
hotels_df.drop_duplicates(keep='first', inplace=True, ignore_index=True)

In [56]:
    #Insert entries into MongoDB
    for index, row in hotels_df.iterrows():

        db.hotels.insert_one(
            {
                'Name': f"{hotels_df.loc[index, 'Name']}",
                'Address': f"{hotels_df.loc[index, 'Address']}",
                'Status': f"{hotels_df.loc[index, 'Status']}",
                'Lattitude': f"{hotels_df.loc[index, 'Latitude']}",
                'Longitude': f"{hotels_df.loc[index, 'Longitude']}"

            }
        )

In [11]:
#Convert the data to a json
with open("data.json", "w") as outfile: 
    json.dump(response, outfile) 

In [31]:
 
# database  
locations_mdb = client["locations_mdb"] 
   
# Created or Switched to collection  
# names: GeeksForGeeks 
Collection = locations_mdb["data"] 
  
# Loading or Opening the json file 
with open('data.json') as file: 
    file_data = json.load(file) 
      
# Inserting the loaded data in the Collection 
# if JSON contains data more than one entry 
# insert_many is used else insert_one is used 
if isinstance(file_data, list): 
   db.data.insert_many(file_data)   
else: 
    db.data.insert_many(file_data) 

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [13]:
for item in response["results"]:
    locations_mdb.data.insert_one(item)


NameError: name 'locations_mdb' is not defined

In [ ]:
for item in response["results"]:
    locations_mdb.data.insert_one(item)

In [62]:
response["results"]

ONAL',
  'geometry': {'location': {'lat': 37.7771861, 'lng': -122.4231806},
   'viewport': {'northeast': {'lat': 37.7785474802915,
     'lng': -122.4217717197085},
    'southwest': {'lat': 37.7758495197085, 'lng': -122.4244696802915}}},
  'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
  'name': 'Hayes Valley Inn',
  'opening_hours': {'open_now': True},
  'photos': [{'height': 2160,
    'html_attributions': ['<a href="https://maps.google.com/maps/contrib/114331831145864172176">Bert Bräutigam</a>'],
    'photo_reference': 'ATtYBwLYiTp-lkrFCY7PIhx2HWnvJ90UNS7Jmn5DpK-eTKnYG-ioNc_vEM79RE1F73P45TSFqd97C8JWBptNLLOOj8aXUn1-X_ocK9tWMHgQCai2IgoCAvwRzDA3OYaZ9voUs0_JL34xHmrWm-TZT1YOFb8AhoLluNVIqDzl3dhjkXIArDwJ',
    'width': 3840}],
  'place_id': 'ChIJAQ2zvpiAhYARTcxfopA7klQ',
  'plus_code': {'compound_code': 'QHGG+VP Central Embarcadero Piers Historic District, CA, USA',
   'global_code': '849VQHGG+VP'},
  'rating': 3.9,
  'reference': 'ChIJAQ2zvpiAhYARTcxfo

In [ ]:
#Retrieve your data